In [3]:
import numpy as np
from numpy import asarray
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import scipy.io as spio
import pickle
import glob
import argparse
from PIL import Image
import pandas as pd

In [4]:
# data paths
train_data_dir = r'C:\Users\prath\Downloads\Machine_Learning\Dogs\train'
test_data_dir = r'C:\Users\prath\Downloads\Machine_Learning\Dogs\test'
val_data_dir = r'C:\Users\prath\Downloads\Machine_Learning\Dogs\valid'
csv_path = r'C:\Users\prath\Downloads\Machine_Learning\Dogs\dogs.csv'

In [5]:
# create image path array for above dirs
def create_image_path_array(path):
    folder_paths = [path+'\\'+each for each in os.listdir(path)]
#     print(len(folder_paths))
    classes = [each for each in os.listdir(path)]
#     print(classes)
    image_paths = []
    for folder in folder_paths:
        image_paths = image_paths + [folder+'\\'+img for img in os.listdir(folder)]
    return image_paths

In [6]:
# create separate image path arrays for train, test, valid
train_image_paths = create_image_path_array(train_data_dir)
test_image_paths = create_image_path_array(test_data_dir)
val_image_paths = create_image_path_array(val_data_dir)

In [7]:
# one hot encoding given csv file with labels and test,train,valid marked
def one_hot_encoding(csv_path):
    df = pd.read_csv(csv_path)
    one_hot = pd.get_dummies(df.labels)
    class_names = pd.unique(df['labels'])
#     print(class_names)
    one_hot = one_hot.iloc[:,:-1]
    class_names = pd.unique(df['labels'])
    df = df.drop('filepaths', axis = 1).drop('labels', axis = 1).join(one_hot)
    ytrain = df[df['data set'] == 'train'].drop('data set', axis = 1).to_numpy()
    ytest = df[df['data set'] == 'test'].drop('data set', axis = 1).to_numpy()
    yval = df[df['data set'] == 'valid'].drop('data set', axis = 1).to_numpy()
    return ytrain, ytest, yval

In [8]:
ytrain, ytest, yval = one_hot_encoding(csv_path)

In [9]:
# [TO DO]
# resize the images or apply some dimensionality reduction technique as needed.
# we might also need to do some data augmentation here.
def image_processing(image_path_array):
    x = []
    for image in image_path_array:
        img = cv2.imread (image, cv2.IMREAD_COLOR)
#         img = cv2.resize(img, (64,64))
        x.append(img)
    return x

In [10]:
xtrain = image_processing(train_image_paths)
xtest = image_processing(test_image_paths)
xval = image_processing(val_image_paths)

In [11]:
# saving this as an npz file just in case it takes too long to run
np.savez('Dogs_npz', xtrain, ytrain, xtest, ytest, xval, yval)